# 39AA Project Part 3
[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/glaframb71/CS39AA-Project/blob/main/39aa-project-part-2.ipynb)

<font size='3'>Welcome to the third and final part of my 39AA Project! In the last two parts, we imported a data set of NFL players from the 2022 season that included their 'nflId', 'height', 'weight', 'birthDate', 'collegeName', 'position', and 'displayName'. With that data set, I decided I wanted to try and train some models to predict and athletes NFL position based of their 'height', 'weight', and 'collegeName'. Exploratory Data Analysis was then performed to get a visual representation of the data we had to work with and to better understand the structure of the set. We then moved on to train a RandomForestClassifier ML model with the data to see how well it would perform. In Part 3, we will be doing similar work that was done in Part 2 but this time we will be attempting to train an exisiting deep learning model. Let's go ahead and get started shall we?</font>

<font size='3'>Let's start by importing the neccesary libraries that are needed to complete this task.</font>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

<font size='3'>Now let's import our dataset.</font>

In [2]:
playersDataUrl = "https://raw.githubusercontent.com/glaframb71/CS39AA-Project/main/data/players.csv"
playersData = pd.read_csv(playersDataUrl)

playersData.head()

,nflId,height,weight,birthDate,collegeName,position,displayName
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady
1,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters
2,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers
3,30842,6-6,267,1984-05-19,UCLA,TE,Marcedes Lewis
4,33084,6-4,217,1985-05-17,Boston College,QB,Matt Ryan


<font size='3'>The dataset has been successfully imported! Let's clean it up and filter out the players we had filtered out in Part 2.</font>

In [3]:
playersData = playersData.copy()
playersData.loc[:, 'height_inches'] = playersData['height'].apply(lambda x: int(x.split('-')[0]) * 12 + int(x.split('-')[1]))
height_index = playersData.columns.get_loc('height')
playersData.insert(height_index + 1, "height_inches", playersData.pop("height_inches"))
players = playersData[['height_inches', 'weight','collegeName', 'position']]

positions_to_drop = ['DB', 'LS']
filtered_players = players[~players['position'].isin(positions_to_drop)]

filtered_players.head()

,height_inches,weight,collegeName,position
0,76,225,Michigan,QB
1,76,328,Arkansas,T
2,74,225,California,QB
3,78,267,UCLA,TE
4,76,217,Boston College,QB


<font size='3'>We have our cleaned up set with the collumns 'height_inches', 'weight', 'collegeName', and 'position' that represent NFL players. Now lets split this set up into our X and y data set.</font>

X = filtered_players.drop('position', axis=1)
y = filtered_players['position']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=10)

X_train_encoded = pd.get_dummies(X_train, columns=['collegeName'])

encoded_columns = X_train_encoded.columns

X_val_encoded = pd.get_dummies(X_val, columns=['collegeName'])

missing_cols = set(X_train_encoded.columns) - set(X_val_encoded.columns)

missing_cols_df = pd.DataFrame(False, index=X_val_encoded.index, columns=list(missing_cols))

X_val_encoded = pd.concat([X_val_encoded, missing_cols_df], axis=1)

X_val_encoded = X_val_encoded[encoded_columns]

In [4]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# Assuming df is your DataFrame with the player data
X = filtered_players[['height_inches', 'weight', 'collegeName']]
y = filtered_players['position']

# Preprocess the data
column_transformer = ColumnTransformer([
    ('num', StandardScaler(), ['height_inches', 'weight']),
    ('cat', OneHotEncoder(sparse_output=False), ['collegeName'])  # Use sparse_output=False
], remainder='passthrough')

X = column_transformer.fit_transform(X)
y = OneHotEncoder(sparse_output=False).fit_transform(y.to_frame())  # Use sparse_output=False

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))  # Output layer

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy}')


Epoch 1/50
38/38 [==============================] - 1s 10ms/step - loss: 2.7632 - accuracy: 0.1538 - val_loss: 2.6664 - val_accuracy: 0.2889
Epoch 2/50
38/38 [==============================] - 0s 3ms/step - loss: 2.5093 - accuracy: 0.2564 - val_loss: 2.3726 - val_accuracy: 0.2889
Epoch 3/50
38/38 [==============================] - 0s 3ms/step - loss: 2.1854 - accuracy: 0.2713 - val_loss: 2.0946 - val_accuracy: 0.3852
Epoch 4/50
38/38 [==============================] - 0s 4ms/step - loss: 1.9518 - accuracy: 0.3449 - val_loss: 1.9275 - val_accuracy: 0.4000
Epoch 5/50
38/38 [==============================] - 0s 3ms/step - loss: 1.7844 - accuracy: 0.3863 - val_loss: 1.8047 - val_accuracy: 0.3926
Epoch 6/50
38/38 [==============================] - 0s 3ms/step - loss: 1.6649 - accuracy: 0.4400 - val_loss: 1.7261 - val_accuracy: 0.4222
Epoch 7/50
38/38 [==============================] - 0s 3ms/step - loss: 1.5698 - accuracy: 0.4599 - val_loss: 1.6655 - val_accuracy: 0.4222
Epoch 8/50
38/38 [=